# Tutorial 02: Agent with Tools - Adding Real-World Capabilities

##  Learning Objectives
By the end of this notebook, you will:
- Understand what tools are and why they're essential
- Create Python functions that agents can call
- Use type annotations and docstrings for tool definitions
- Add tools to agents at creation time and runtime
- See how agents decide when to use tools

##  Key Concepts

### What are Tools?
Tools are **Python functions** that agents can call to:
- Get real-time information (weather, news, prices)
- Perform calculations (currency conversion, distance)
- Access external APIs (flights, hotels, restaurants)
- Interact with databases or files

### Function Calling Flow
```
User: "What's the weather in Paris?"
  ↓
Agent: "I need to call the weather tool"
  ↓
Tool: get_weather("Paris") → "Sunny, 22°C"
  ↓
Agent: "The weather in Paris is sunny with a high of 22°C"
```

---

## Step 1: Setup

Import what we need, including Pydantic for better type hints.

In [ ]:
import asyncio
import os
from datetime import datetime
from random import randint, choice
from typing import Annotated

from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from agent_framework.azure import AzureOpenAIChatClient
from pydantic import Field
from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME")

print(" Imports successful!")

✅ Imports successful!


## Step 2: Create Your First Tool

Let's create a simple weather tool. Notice the important elements:
1. **Docstring** - Describes what the function does (the agent sees this!)
2. **Type annotations** - Define parameter types
3. **Field descriptions** - Provide context for the agent
4. **Return type** - What the function returns

In [7]:
def get_weather(
    location: Annotated[str, Field(description="The city or location to get weather for. E.g., 'Paris', 'Tokyo', 'New York'.")],
) -> str:
    """
    Get the current weather conditions for a specific location.
    Returns temperature in Celsius and general conditions.
    """
    # In a real app, this would call a weather API
    # For now, we'll simulate it
    conditions = ["sunny", "partly cloudy", "cloudy", "rainy", "stormy"]
    temp = randint(10, 35)
    condition = choice(conditions)
    
    return f"The weather in {location} is {condition} with a temperature of {temp}°C."

# Test the function directly
print(get_weather("Paris"))

The weather in Paris is stormy with a temperature of 20°C.


## Step 3: Create an Agent with Tools

Now let's give our Travel Assistant the ability to check weather!

In [ ]:
# Create the agent with the weather tool
travel_agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    deployment_name=MODEL_DEPLOYMENT_NAME
),
    name="TravelAssistant",
    instructions="""
    You are a helpful travel assistant. You can provide destination recommendations,
    travel tips, and check current weather conditions.
    
    When users ask about weather, use the get_weather tool to provide accurate information.
    """,
    tools=[get_weather]  # Add tools here!
)

print(" Agent created with weather tool!")

✅ Agent created with weather tool!


## Step 4: Test Tool Usage

Let's ask about weather and see the agent use the tool.

In [10]:
query = "What's the weather like in Barcelona? I'm thinking of visiting."

print(f"User: {query}\n")
response = await travel_agent.run(query)
print(f"Agent: {response}")

User: What's the weather like in Barcelona? I'm thinking of visiting.

Agent: Barcelona is currently experiencing partly cloudy weather with a warm temperature of 34°C. If you enjoy sunny and warm conditions, it's a great time to visit! Be sure to stay hydrated and wear sunscreen while exploring the city. Let me know if you’d like tips or recommendations for your trip.
Agent: Barcelona is currently experiencing partly cloudy weather with a warm temperature of 34°C. If you enjoy sunny and warm conditions, it's a great time to visit! Be sure to stay hydrated and wear sunscreen while exploring the city. Let me know if you’d like tips or recommendations for your trip.


🎉 **Notice**: The agent automatically called `get_weather("Barcelona")` and incorporated the result into its response!

## Step 5: Add More Tools

Let's add a currency converter to help with travel budgeting.

In [11]:
def convert_currency(
    amount: Annotated[float, Field(description="The amount to convert")],
    from_currency: Annotated[str, Field(description="Source currency code (e.g., 'USD', 'EUR', 'GBP')")],
    to_currency: Annotated[str, Field(description="Target currency code (e.g., 'USD', 'EUR', 'GBP')")],
) -> str:
    """
    Convert an amount from one currency to another.
    Uses approximate exchange rates.
    """
    # Simplified exchange rates (in a real app, use a live API)
    rates = {
        "USD": 1.0,
        "EUR": 0.92,
        "GBP": 0.79,
        "JPY": 149.50,
        "CAD": 1.36,
        "AUD": 1.53,
    }
    
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in rates or to_currency not in rates:
        return f"Sorry, I don't have exchange rates for {from_currency} or {to_currency}."
    
    # Convert to USD first, then to target currency
    usd_amount = amount / rates[from_currency]
    converted = usd_amount * rates[to_currency]
    
    return f"{amount} {from_currency} is approximately {converted:.2f} {to_currency}"

# Test it
print(convert_currency(100, "USD", "EUR"))

100 USD is approximately 92.00 EUR


## Step 6: Add a Flight Search Tool

Let's simulate a flight search (in real apps, this would call an API like Amadeus or Skyscanner).

In [12]:
def search_flights(
    origin: Annotated[str, Field(description="Departure city or airport code")],
    destination: Annotated[str, Field(description="Arrival city or airport code")],
    date: Annotated[str, Field(description="Travel date in YYYY-MM-DD format")],
) -> str:
    """
    Search for available flights between two locations on a specific date.
    Returns a summary of available flights with prices.
    """
    # Simulate flight search results
    airlines = ["United", "Delta", "American", "Air France", "British Airways", "Lufthansa"]
    
    flights = []
    for i in range(3):
        airline = choice(airlines)
        price = randint(200, 1200)
        duration = randint(2, 15)
        flights.append(f"- {airline}: ${price} ({duration}h {randint(0, 59)}m)")
    
    result = f"Found {len(flights)} flights from {origin} to {destination} on {date}:\n"
    result += "\n".join(flights)
    return result

# Test it
print(search_flights("New York", "London", "2025-11-15"))

Found 3 flights from New York to London on 2025-11-15:
- Delta: $1114 (10h 43m)
- United: $469 (8h 40m)
- United: $678 (14h 29m)


## Step 7: Create an Enhanced Travel Agent

Now let's create an agent with ALL our tools!

In [ ]:
# Create a more capable agent
enhanced_agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    deployment_name=MODEL_DEPLOYMENT_NAME
),
    name="EnhancedTravelAssistant",
    instructions="""
    You are an advanced travel assistant with access to real-time tools.
    
    Your capabilities:
    - Check current weather conditions
    - Convert currencies for budget planning
    - Search for flight options
    
    When users ask about practical travel information, use the appropriate tools.
    Combine tool results with your knowledge to give comprehensive advice.
    """,
    tools=[get_weather, convert_currency, search_flights]
)

print(" Enhanced agent created with 3 tools!")

✅ Enhanced agent created with 3 tools!


## Step 8: Test Multi-Tool Queries

Let's ask questions that require multiple tools.

In [15]:
# Query that needs weather and currency
query = "I'm planning a trip to Tokyo. What's the weather like there, and how much is 500 USD in Japanese Yen?"

print(f"User: {query}\n")
response = await enhanced_agent.run(query)
print(f"Agent: {response}")

User: I'm planning a trip to Tokyo. What's the weather like there, and how much is 500 USD in Japanese Yen?

Agent: Currently, Tokyo is experiencing partly cloudy weather with a temperature of 20°C—quite pleasant for exploring the city.

For budgeting, 500 USD is approximately 74,750 Japanese Yen (JPY). Let me know if you need more details for your Tokyo trip!
Agent: Currently, Tokyo is experiencing partly cloudy weather with a temperature of 20°C—quite pleasant for exploring the city.

For budgeting, 500 USD is approximately 74,750 Japanese Yen (JPY). Let me know if you need more details for your Tokyo trip!


In [16]:
# Query that needs all tools
query = """
I want to fly from San Francisco to Paris on December 1st, 2025. 
Can you show me flight options, tell me the weather there, and convert 1000 USD to Euros?
"""

print(f"User: {query}\n")
response = await enhanced_agent.run(query)
print(f"Agent: {response}")

User: 
I want to fly from San Francisco to Paris on December 1st, 2025. 
Can you show me flight options, tell me the weather there, and convert 1000 USD to Euros?


Agent: Here’s the information you requested for your trip from San Francisco to Paris on December 1st, 2025:

Flight Options:
- United: $354 (9h 57m)
- Air France: $719 (7h 29m)
- British Airways: $919 (9h 26m)

Weather in Paris:
- Cloudy, around 25°C.

Currency Conversion:
- 1,000 USD is approximately 920 EUR.

Let me know if you need hotel options, tips for Paris, or help booking!
Agent: Here’s the information you requested for your trip from San Francisco to Paris on December 1st, 2025:

Flight Options:
- United: $354 (9h 57m)
- Air France: $719 (7h 29m)
- British Airways: $919 (9h 26m)

Weather in Paris:
- Cloudy, around 25°C.

Currency Conversion:
- 1,000 USD is approximately 920 EUR.

Let me know if you need hotel options, tips for Paris, or help booking!


##  Understanding Tool Execution

Here's what happens behind the scenes:

1. **Agent analyzes the query** - Determines which tools are needed
2. **Tool calls are made** - Functions execute with extracted parameters
3. **Results are collected** - Tool outputs are gathered
4. **Agent synthesizes response** - Combines tool results with context

The agent can call:
- **Zero tools** - If the query doesn't need them
- **One tool** - For simple requests
- **Multiple tools** - In sequence or parallel
- **Same tool multiple times** - If needed (e.g., weather for multiple cities)

## Step 9: Tools at Runtime

You can also add tools to specific queries instead of the agent.

In [18]:
# Create an agent without tools
basic_agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    deployment_name=MODEL_DEPLOYMENT_NAME
),
    name="BasicAgent",
    instructions="You are a helpful travel assistant."
    # No tools defined here!
)

# Add tools only for this specific query
query = "What's the weather in Rome?"
response = await basic_agent.run(query, tools=[get_weather])

print(f"User: {query}")
print(f"Agent: {response}")

User: What's the weather in Rome?
Agent: The weather in Rome is currently cloudy with a temperature of 15°C.


## Step 10: Tool with Error Handling

Let's create a more robust tool that handles errors gracefully.

In [19]:
def get_travel_advisory(
    country: Annotated[str, Field(description="The country to check travel advisories for")],
) -> str:
    """
    Get current travel advisory level for a country.
    Returns safety information and recommendations.
    """
    try:
        # Simulate advisory data
        advisories = {
            "France": "Level 1: Exercise normal precautions",
            "Japan": "Level 1: Exercise normal precautions",
            "Mexico": "Level 2: Exercise increased caution",
            "Egypt": "Level 2: Exercise increased caution",
            "Afghanistan": "Level 4: Do not travel",
        }
        
        country = country.strip().title()
        
        if country in advisories:
            return f"Travel advisory for {country}: {advisories[country]}"
        else:
            return f"No specific travel advisory found for {country}. Please check official sources."
            
    except Exception as e:
        return f"Error retrieving travel advisory: {str(e)}"

# Test it
print(get_travel_advisory("Japan"))
print(get_travel_advisory("Atlantis"))  # Non-existent country

Travel advisory for Japan: Level 1: Exercise normal precautions
No specific travel advisory found for Atlantis. Please check official sources.


## 💡 Best Practices for Tools

1. **Clear docstrings** - The agent uses these to understand when to call the tool
2. **Type annotations** - Help the agent extract correct parameters
3. **Field descriptions** - Provide examples and context
4. **Error handling** - Return helpful messages instead of crashing
5. **Consistent returns** - Always return strings (or use structured output)
6. **Descriptive names** - Function names should clearly indicate purpose

### ❌ Bad Tool
```python
def weather(x):
    return api.call(x)
```

### ✅ Good Tool
```python
def get_current_weather(
    location: Annotated[str, Field(description="City name or coordinates")],
) -> str:
    """Get real-time weather for a location."""
    try:
        # Implementation
        pass
    except Exception as e:
        return f"Error: {e}"
```

##  Practice Exercises

Try creating these tools:

1. **Hotel search tool** - Search for hotels in a city with price range
2. **Distance calculator** - Calculate distance between two cities
3. **Restaurant finder** - Find restaurants by cuisine type in a location
4. **Time zone converter** - Convert time between different zones

In [20]:
# Exercise 1: Create a hotel search tool
def search_hotels(
    city: Annotated[str, Field(description="City to search for hotels")],
    max_price: Annotated[int, Field(description="Maximum price per night in USD")],
) -> str:
    """
    Search for hotels in a city within a budget.
    """
    # Your implementation here
    pass

# Test your tool!

##  Key Takeaways

1. **Tools extend agent capabilities** beyond just LLM knowledge
2. **Agents intelligently decide** when and how to use tools
3. **Function signatures matter** - Good annotations = better tool usage
4. **Multiple tools can work together** to solve complex queries
5. **Tools can be added** at agent creation or per-query

##  Still Missing

Our agent still:
-  Doesn't remember previous conversations
-  Can't maintain context across queries
-  Starts fresh with each `run()` call

**In the next tutorial**, we'll add **Threads** for stateful, multi-turn conversations!

---